# Jupyter Notebook для тестирования задач в регионах

# 1. Подготовка

### Импортируем библиотеки

In [ ]:
import requests
import pathlib
import client_lib # импортируем библиотеку для работы с ML Space

### Устанавливаем переменные

In [ ]:
BASE_DIR = str(pathlib.Path().absolute())
print(f"Working dir: {BASE_DIR}")

# 2. Запуск задачи обучения

Класс client_lib.Job() позволяет запускать распределённые задачи в регионе. 

Обязательные параметры для запуска задачи обучения:
- **script** – путь к запускаемому скрипту
- **base_image** – базовый образ, в котором будет исполняться скрипт обучения модели
- **instance_type** – конфигурация вычислительных ресурсов, используемых для решения задач

Подробное описание параметров можно найти в документации по [ссылке](https://cloud.ru/ru/docs/aicloud/mlspace/concepts/client-lib__job.html)

По умолчанию задачи запускаются в регионе Christofari.V100. Для того, чтобы запустить задачу в другом регионе, необходимо указать регион в параметре region.

Доступные регионы и их обозначения в client_lib:

- Christofari.V100 – DGX2-MT
- Christofari.A100 – A100-MT
- Cloud.Region.A100 (GPU Tesla A100) – SR002-MT
- Cloud.Region.HP1 – SR003
- Cloud.Region.HP- DGX2-MT – SR006

Для примера запустим задачу в регионе Cloud.Region.A100 (SR002-MT)

Для масштабирования задачи доступны следующие параметры:

- **n_workers** – количество рабочих узлов региона, на котором будет исполняться скрипт
- **instance_type** – конфигурация вычислительных ресурсов, используемых для решения задач

Для выбора значения параметра instance_type воспользуемся методом get_instance_types(). Подробнее об использовании метода в [документации](https://cloud.ru/ru/docs/aicloud/mlspace/concepts/client-lib__common-methods.html#client-lib-get-instance-types)

Выведем доступные значения instance_type для региона SR002-MT

In [ ]:
client_lib.get_instance_types(client_lib.ClusterType.MT).query('region == "SR002-MT"')

Для примера запустим задачу на 1 воркере с 1 ГПУ.

Сохраним в переменные название региона, instanse_type и образы

In [ ]:
REGION = "SR002-MT"
INSTANCE_TYPE = "a100.1gpu.40"
N_WORKERS = 1
BASE_IMAGE = "cr.ai.cloud.ru/aicloud-base-images/cuda12.1-torch2-py39:0.0.36"

In [ ]:
job = client_lib.Job(
    base_image=BASE_IMAGE,
    script=f"{BASE_DIR}/train_distributed_example-torch2.py",
    region=REGION,
    instance_type=INSTANCE_TYPE,
    n_workers=N_WORKERS,
    type="pytorch2",
    processes_per_worker=1,
    job_desc="pytorch2 | client_lib | use_env=False | torch2",
)


Запустим задачу методом submit()

In [ ]:
job.submit()

Для получения статуса задачи воспользуемся методом status()

Возможные статусы задачи 

- «Pending» - Задача находится в очереди на выделение ресурсов, которые нужны для ее исполнения.

- «Running» - Задача обучения выполняется.

- «Completed» или «Succeeded» – Задача обучения завершилась.

- «Completing» – Задача обучения завершается.

- «Failed» – Задача обучения завершилась с ошибкой, рекомендуется проверить логи задачи.

- «Deleted» или «Terminated» – Задача обучения удалена.

- «Stopped» или «Aborted» – Задача обучения остановлена.

- «Terminating» – Задача обучения останавливается. Освобождаются ресурсы, задача и поды удаляются.

- «Aborting» – Задача обучения останавливается. Освобождаются ресурсы, удаляются только поды.

In [ ]:
job.status()

Для просмотра логов задачи можно вызвать метод logs()

Логи будут доступны после запуска задачи(перехода в статус Running)

In [ ]:
job.logs()

Задача завершиться автоматически после выполнения скрипта. Если требуется прервать выполнение задачи, можно воспользоваться методом kill()

In [ ]:
job.kill()

### Передача переменных окружения и флагов при запуске задачи

В параметре flags можно передать флаги, с которыми необходимо запустить скрипт. Параметр принимает словарь в формате {"<флаг>": "<значение>"}

Пример задания параметра flags:

```
flags={
    "foo": "foo_value", 
    "bar": "bar_value",
}
```
Скрипт будет запущен с параметрами ```<your_script> --batch_size=512 --model="mymodel50" --xla=False



Переменные окружения можно передать в параметре env_variables. Параметр принимает словарь в формате {"<название переменной>": "<значение>"}


Пример задания параметра env_variables:

```
env_variables={
    "BAZ": "baz_value",
    "QUUX": "quux_value",
}
```

In [ ]:
job = client_lib.Job(
    base_image=BASE_IMAGE,
    script=f"{BASE_DIR}/env_variables_flags_test.py",
    region=REGION,
    instance_type=INSTANCE_TYPE,
    n_workers=1,
    type="pytorch2",
    processes_per_worker=1,
    flags={
        "foo": "foo_value", 
        "bar": "bar_value",
    },
    env_variables={
        "BAZ": "baz_value",
        "QUUX": "quux_value",
    },
    job_desc="testing flags and env_variables",
    pytorch_use_env=True,
    in_communal_cluster=True,
)

job.submit()

In [ ]:
job.status()

Проверим в логах, что флаги и переменные окружения передались при запуске задачи

In [ ]:
job.logs()

### Подключение к задаче по SSH

**Подключение доступно в Jupyter server с образами версии 0.0.95 и выше**

Для примера запустим бесконечную задачу. Чтобы задача не завершилась, укажем ```script="sleep infinity"``` и ```type="binary"```

In [ ]:
job = client_lib.Job(
    base_image=BASE_IMAGE,
    script="sleep infinity", # передаём sleep infinity для того, чтобы задача не завершалась
    region=REGION,
    instance_type="a100plus.1gpu.80vG.12C.96G",
    n_workers=1,
    type="binary", # передаём тип binary для запуска shell-скриптов
    processes_per_worker=1,
    job_desc="sleep infinity",
    in_communal_cluster=True
)

job.submit()

Для подключения нам нужно будет имя задачи, получим его с помощью атрибута job_name

In [ ]:
job.job_name

In [ ]:
job.status()

Подключится можно к задаче в статусе "Running"

Для подключения можно использовать команду mlspace ssh. Есть 2 варианта подключения – по хосту и local rank.

#### Подключение по хосту
Для подключения по хосту необходимо выполнить в терминале команду ```mlspace ssh by-host HOST```

В качестве хоста необходимо указать 
- ```lm-mpi-job-<uuid_v4>-mpimaster-0``` – для подключения к мастеру 
- ```lm-mpi-job-<uuid_v4>-mpiworker-<number>``` – для подключения к воркеру                   
                                                                                                                              
Пример использования: ```mlspace ssh by-host lm-mpi-job-842ec184-4610-420e-9ca8-8198ddf9167e-mpiworker-1```                                                       

#### Подключение по local rank
Для подключения по local rank необходимо выполнить в терминале команду ```mlspace ssh by-rank --rank <number> JOB_NAME```

в параметр --rank передать положительное число worker_(number). 
Значения:
- rank 0 - mpimaster-0
- rank 1 - mpiworker-0
- rank N - mpiworker-{N + 1}
одключение через ssh к заданому worker_(number).                                                                                                             
 Пример использования:mlspace ssh by-rank lm-mpi-job-842ec184-4610-420e-9ca8-8198ddf9167e --rank 1                                                             


**Важно** после проверки подключения необходимо завершить задачу

In [ ]:
job.kill()

In [ ]:
client_lib.jobs(region=f"{REGION}")

# 3. [Optional] - Сохранение промежуточных результатов обучения модели

Если в процессе обучения модели пользователь сохраняет промежуточные результаты (checkpoints) обучения, они попадают в папку `./logs`. Их можно скачать через веб-интерфейс Jupyter-ноутбука или скопировать из локально доступной файловой системы в хранилище S3.

## Выгрузка результатов обучения модели с NFS на S3

Для переноса файлов между NFS и S3 можно использовать [методы копирования client_lib](https://aicdoc-613-add-in-faq-from--e42921f5.docs.sbercloud.dev/aicloud/mlspace/concepts/client-lib__copy-to-nfs.html#id1) или правила переноса [Data Transfer Service](https://cloud.ru/ru/docs/aicloud/mlspace/concepts/guides/guides__dc/data-catalog__data-processing__create-transfer-rule.html)

Рассмотрим копирование файлов из NFS на S3 воркспейса с помощью метода ```copy_from_nfs()```



In [ ]:
relative_path = str(pathlib.Path().absolute().relative_to(pathlib.Path().absolute().parent))

In [ ]:
client_lib.copy_from_nfs(
    source_path=f"{relative_path}/logs/", # укажем путь к папке logs без /home/jovyam 
    from_region=client_lib.RegionEnum.SR002_MT, # укажем регион, в нашем случае SR002-MT
    destination_path="quck-start" # укажем место назначения переноса
)

С помошью ID посмотрим логи переноса

In [ ]:
client_lib.get_transfer_data_logs("3ebe26ec-1d6f-4797-926e-642f1fcbe12f") # id переноса берём из вывода предыдущей ячейки

В результате мы перенесли папку logs в S3 воркспейса в папку quick-start.

Проверить наличие файлов на S3 можно перейдя в раздел "Data Catalog" -> "Объектное Хранилище" и выбрав бакет воркспейса с доступом "Public".

# 4. [Optional] - Собираем кастомный образ с нужными библиотеками

##### Посмотреть содержимое файла requirements.txt:

In [ ]:
%cat ./requirements.txt

##### Запуск сборки кастомного образа с необходимыми библиотеками

После выполнения этой задачи собранный образ должен оказаться в docker registry

**Важно! Для сборки кастомного образа файл requirements.txt должен быть загружен на NFS региона Christofari.V100**


Список базовых образов, которые нужно указать в зависимости от региона в параметре "from_image": [Образы для задач обучения](https://cloud.ru/ru/docs/aicloud/mlspace/concepts/environments__basic-images-list__jobs.html)




Также есть возможность собрать кастомный образ локально с помощью Docker. Инструкция доступна по [ссылке](https://cloud.ru/ru/docs/aicloud/mlspace/concepts/guides/guides__mt/environments__docker-registry__custom__job__image.html)

In [ ]:
job = client_lib.ImageBuildJob(
    from_image=BASE_IMAGE,  # базовый образ для задач обучения
    requirements_file=f"{BASE_DIR}/requirements.txt", # файл с зависимостями для кастомного образа
)

job.submit()

In [ ]:
job.new_image  # идентификатор кастомного образа

In [ ]:
job.logs()  # просмотр логов сборки образа в интерактивном режиме